# Compare Greedy Algorithm to Degree Centrality
This code goes through different numbers of starting set sizes and computes both the most degree central and the greedy approximated starting set. It then runs the simulation on both of them seperately and compares the resulting cascade sizes. Additionaly, it applies the function that the greedy algorithm uses to estimate the cascade size on both sets. Then for each size, the mean over multiple samples is plotted.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from copy import deepcopy

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

from model.agent import Agent
from model.world import World

from model.utils import *
from model.news import News
from model.agent import AgentState

In [2]:
def cascadesize(world):
    return sum([1 if a.is_active() else 0 for a in world.agents.values()])

def measure_cascadesize(world, starting, max_iter):
    for a in starting.keys():
        a.states[starting[a]] = AgentState.ACTIVE
    for iter in range(1, max_iter):
        x = world.update(verbose=True)
        if x == 0:
            break
    return cascadesize(world)

In [3]:
def comp_starting_sets(worlds, sets, max_iter = 20):
    ccs = []
    for s, w in zip(sets, worlds):
        ccs.append(measure_cascadesize(w, s, max_iter))
    return ccs

In [4]:
def compute(sizes, samples = 5):
    """
    takes very long to run!
    """
    c_greedy = [0 for _ in range(len(sizes))]
    c_degree = [0 for _ in range(len(sizes))]
    est_greedy = [0 for _ in range(len(sizes))]
    est_degree = [0 for _ in range(len(sizes))]
    i = 0
    for k in sizes:
        for _ in range(samples):
            sensation_news = 0.5
            decay_parameter_news = 0
            n1 = News(0, sensation_news, decay_parameter_news)
            news1 = {n1.name: n1}
            number_agents = 200
            names_agents = [_ for _ in range(number_agents)]
            threshold = np.random.random(number_agents)
            independence = np.random.random(number_agents)
            w1 = construct_world(names_agents, threshold, independence, news1)
            w2 = deepcopy(w1)
            kset = approx_most_influential(w1, k, sample_size=25, verbose=False) #THIS TAKES SOME TIME !!!
            s1 = {}
            for agent in kset:
                s1[agent] = 0
            degree_centrality = []
            for _ in range(k):
                best = None
                bdeg = -1
                for a in w2.graph.nodes():
                    if w2.agents[a] in degree_centrality:
                        continue
                    d = w2.graph.out_degree(a)
                    if d > bdeg:
                        bdeg = d
                        best = w2.agents[a]
                degree_centrality.append(best)
            s2 = {}
            for agent in degree_centrality:
                s2[agent] = 0
            print([a.name for a in kset])
            print([a.name for a in degree_centrality])
            #reapplying the estimation for the influence used in the greedy algorithm to the solutions
            est_greedy[i] += get_expected_number_of_influenced_agents(w1, [a.name for a in kset], 100) / np.float64(samples)
            est_degree[i] += get_expected_number_of_influenced_agents(w2, [a.name for a in degree_centrality], 100) / np.float64(samples)
            n = 50
            c1, c2 = comp_starting_sets([w1, w2], [s1, s2], n)
            c_greedy[i] += c1 / samples
            c_degree[i] += c2 / samples
        print(k)
        i += 1
    return (c_greedy, c_degree, est_greedy, est_degree)

In [5]:
s = [1,3,5,10,15] #,20,30] less sizes so that it runs faster
c_greedy, c_degree, est_greedy, est_degree = compute(s)

[115]
[0]
[3]
[3]
[15]
[0]
[8]
[8]
[0]
[3]
1
[8, 23, 4]
[8, 4, 9]
[8, 10, 133]
[8, 10, 0]
[0, 1, 3]
[1, 3, 0]
[0, 49, 17]
[0, 17, 3]
[1, 3, 25]
[1, 3, 2]
3
[3, 9, 46, 13, 4]
[3, 1, 9, 4, 0]
[1, 106, 4, 11, 173]
[0, 1, 8, 3, 4]
[0, 3, 127, 42, 89]
[0, 3, 8, 6, 4]
[173, 2, 18, 3, 142]
[3, 1, 2, 18, 8]
[1, 16, 4, 3, 19]
[1, 16, 3, 4, 17]
5
[0, 17, 3, 72, 76, 96, 59, 4, 154, 33]
[0, 3, 4, 5, 16, 1, 2, 10, 12, 13]
[34, 0, 157, 40, 188, 116, 117, 49, 91, 123]
[3, 0, 5, 25, 2, 26, 4, 16, 40, 1]
[1, 193, 19, 17, 143, 184, 69, 146, 4, 85]
[1, 0, 4, 2, 3, 17, 9, 32, 13, 6]
[1, 189, 4, 70, 133, 25, 108, 99, 199, 3]
[1, 8, 10, 4, 3, 2, 5, 25, 0, 6]
[80, 11, 16, 8, 18, 77, 66, 5, 195, 1]
[3, 4, 8, 11, 5, 1, 17, 18, 2, 10]
10
[3, 26, 156, 17, 172, 5, 158, 88, 33, 129, 126, 7, 2, 120, 178]
[3, 1, 2, 4, 26, 0, 35, 28, 24, 9, 34, 5, 6, 15, 17]
[2, 73, 176, 0, 86, 153, 105, 155, 66, 120, 58, 152, 75, 150, 164]
[2, 8, 3, 11, 0, 10, 5, 6, 18, 24, 25, 12, 14, 75, 4]
[153, 182, 3, 11, 57, 36, 103, 68, 19, 5

In [2]:
print(c_greedy, c_degree, est_greedy, est_degree, sep="\n")

In [3]:
plt.plot(s, c_greedy, "b.", label="greedy")
plt.plot(s, est_greedy, "b+", label="expected cascade size greedy")
plt.plot(s, c_degree, "r.", label="degree centrality")
plt.plot(s, est_degree, "r+", label="expected cascade size degree centrality")
plt.grid()
plt.legend()
plt.show()